__собираем предложения о продаже домов__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-21_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-22_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-16_15-24_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-19_12-40_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-22_09-59_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-09-26_11-23_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-10-08_21-15_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-10-14_15-55_raw_sudak.xlsx',
 'data/raw/house/avito/avito_2022-10-14_15-56_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-11-01_14-02_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-11-04_17-15_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-11-07_11-07_raw_sevastopol.xlsx',
 'data/raw/h

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

27702


In [8]:
data_avito.groupby('place')['ts'].max()

place
bahchisaray   2023-03-29 16:50:38.487
sevastopol    2023-03-29 16:50:22.853
sudak         2023-03-29 16:50:52.064
Name: ts, dtype: datetime64[ns]

In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

27702


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
23798,2451387006,дом 330 м² на участке 8 сот.,36000000,,NaN,"Только продажа! Обмен не предлагать! Земельный уч-к S=792 кв. М. В г. Севастополь, Радиогорка.\nКад. № 91:04:003004:610 (Ижс с правом регистрации).\nМагистральный газ по забору (не подключен).\nОсвоенная территория S=477 кв. М., в заборе. На ней 2-а здания по",23,2023-03-09 12:26:52.320,sevastopol,330.00,False,False,False,8.00,36.00,4-8,300+
977,2480537609,дом 210 м² на участке 6 сот.,15500000,,"ул. Генерала Саймонова, 31","Продаётся напрямую от собственника отличный дом, расположенный на участке 6,5 соток под Ижс в тихом, благополучном и быстроразвивающемся районе Севастополя. Все коммуникации в доме Центральные -газ, вода, электричество и канализация. Дом удачно спланирован",20,2022-09-15 11:53:29.976,sevastopol,210.00,False,False,False,6.00,15.50,4-8,150-300
8557,2365589421,"дом 705,4 м² на участке 8,7 сот.",77990000,,"Готская ул., 18","Продается дом в элитном районе города с развитой инфраструктурой( школы, детские сады, поликлиника, магазины). Все это в шаговой доступности. Удобный выезд из дома во все районы города. Дом построен из экологически чистых материалов( инкерманских блоков).",45,2022-11-01 14:02:31.396,sevastopol,705.40,False,False,False,8.70,77.99,8-20,300+


In [12]:
data_avito.groupby('place')['ts'].max()

place
bahchisaray   2023-03-29 16:50:38.487
sevastopol    2023-03-29 16:50:22.853
sudak         2023-03-29 16:50:52.064
Name: ts, dtype: datetime64[ns]

In [13]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [14]:
assert len(data_avito)>0

In [15]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27702 entries, 0 to 27701
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             27702 non-null  int64         
 1   title                27702 non-null  object        
 2   price                27702 non-null  int64         
 3   obj_name             27702 non-null  object        
 4   adr                  22910 non-null  object        
 5   description          27700 non-null  object        
 6   avito_page           27702 non-null  int64         
 7   ts                   27702 non-null  datetime64[ns]
 8   place                27702 non-null  object        
 9   house_area           27702 non-null  float64       
 10  is_part              27702 non-null  bool          
 11  is_townhouse         27702 non-null  bool          
 12  is_SNT               27702 non-null  bool          
 13  land_area            27702 non-

In [16]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                     True
description             True
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [17]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5076,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",48,2022-09-16 15:21:10.419,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
5213,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",1,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
5384,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-16 15:22:23.263,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
5511,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-16 15:24:24.472,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5645,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,2,2022-09-19 12:40:34.305,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN
5782,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,2,2022-09-22 09:59:12.841,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [18]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [19]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

27702


In [20]:
data_avito.groupby('place')['ts'].max()

place
bahchisaray   2023-03-29 16:50:38.487
sevastopol    2023-03-29 16:50:22.853
sudak         2023-03-29 16:50:52.064
Name: ts, dtype: datetime64[ns]

## обновляем таблицу адресов

In [21]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [22]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].fillna('').progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data[['adr_orig','adr']].sample(10)

  0%|          | 0/27702 [00:00<?, ?it/s]

  0%|          | 0/27702 [00:00<?, ?it/s]

,adr_orig,adr
6443,садоводческое товарищество Мрия,"Севастополь,садоводческое товарищество Мрия"
3661,Гагаринский муниципальный округ,"Севастополь,Гагаринский район"
13337,"пос. городского типа Куйбышево, Колхозная ул.","Бахчисарай,пос. городского типа Куйбышево, Колхозная ул."
24385,NaN,"Севастополь,"
18973,товарищество собственников недвижимости Лукоморье,"Севастополь,товарищество собственников недвижимости Лукоморье"
11945,"садоводческое товарищество Вишенка, пер. Анатолия Дрыгина","Севастополь,садоводческое товарищество Вишенка, пер. Анатолия Дрыгина"
22224,NaN,"Севастополь,"
23891,NaN,"Севастополь,"
11414,Пластунская ул.,"Севастополь,Пластунская ул."
593,"садоводческое товарищество Эллада, 4","Севастополь,садоводческое товарищество Эллада, 4"


In [23]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[1])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2023-03-29 16:54:30 | LocationUpdater: 10012 addresses in index
[INFO    ] 2023-03-29 16:54:30 | LocationUpdater: 10655 addresses total
[INFO    ] 2023-03-29 16:54:30 | LocationUpdater: 10012 addresses defined
[INFO    ] 2023-03-29 16:54:30 | LocationUpdater: 643 addresses undefined


  0%|          | 0/643 [00:00<?, ?it/s]

[INFO    ] 2023-03-29 16:58:58 | LocationUpdater: 643 new addresses found


In [24]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

## дополняем данные геометкой

In [25]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

всего записей: 27702
записей без геометки: 0


In [26]:
# data.loc[ data['adr_orig']=='квартал Старый Город', ['latitude','longitude']] 
# data[ data['adr_orig'].str.match('.*садоводческое товарищество Голубой залив.*') ][['adr','latitude','longitude']]

In [27]:
data.loc[data['adr_orig']=='садоводческое товарищество Голубой залив', ['latitude','longitude']]=None
# data.loc[data['adr_orig'].str.match('.*садоводческое товарищество Голубой залив.*'),
#                   ['latitude','longitude']]=None
data.loc[data['adr_orig']=='квартал Старый Город',['latitude','longitude']]=None

In [28]:
# data['dt']

In [29]:
data.groupby('place')['ts'].max()

place
bahchisaray   2023-03-29 16:50:38.487
sevastopol    2023-03-29 16:50:22.853
sudak         2023-03-29 16:50:52.064
Name: ts, dtype: datetime64[ns]

In [30]:
assert len(data)>0
data.to_pickle(result_file_path)

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27702 entries, 0 to 27701
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   title                27702 non-null  object        
 1   price                27702 non-null  int64         
 2   obj_name             27702 non-null  object        
 3   adr_orig             22910 non-null  object        
 4   description          27700 non-null  object        
 5   avito_page           27702 non-null  int64         
 6   ts                   27702 non-null  datetime64[ns]
 7   place                27702 non-null  object        
 8   house_area           27702 non-null  float64       
 9   is_part              27702 non-null  bool          
 10  is_townhouse         27702 non-null  bool          
 11  is_SNT               27702 non-null  bool          
 12  land_area            27702 non-null  float64       
 13  priceM               27702 non-

In [31]:
data[['ts','place']].groupby(['ts','place']).size()

ts                       place      
2022-09-15 11:53:29.976  sevastopol     2432
2022-09-15 14:42:27.254  bahchisaray     266
2022-09-15 14:47:45.056  sudak           118
2022-09-16 15:21:10.419  sevastopol     2382
2022-09-16 15:22:23.263  bahchisaray     267
2022-09-16 15:24:24.472  sudak           123
2022-09-19 12:40:34.305  sudak           121
2022-09-22 09:59:12.841  sudak           121
2022-09-26 11:23:01.176  sudak           122
2022-10-08 21:15:03.796  sudak           108
2022-10-14 15:55:58.015  sudak           108
2022-10-14 15:56:46.465  bahchisaray     251
2022-11-01 14:02:31.396  sevastopol     2202
2022-11-04 17:15:44.835  sevastopol     2146
2022-11-07 11:07:33.153  sevastopol     2095
2022-11-07 12:08:03.980  bahchisaray     254
2022-11-07 12:08:24.612  sudak           103
2022-11-15 13:31:15.084  sevastopol     2055
2022-11-15 13:31:37.170  bahchisaray     250
2022-11-15 13:31:49.612  sudak           103
2022-11-28 18:22:27.677  sevastopol     2138
2022-11-28 18:24:4